![tracker](https://us-central1-vertex-ai-mlops-369716.cloudfunctions.net/pixel-tracking?path=statmike%2Fvertex-ai-mlops%2FFramework+Workflows%2FCatBoost&file=CatBoost+Custom+Prediction+With+FastAPI.ipynb)
<!--- header table --->
<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/Framework%20Workflows/CatBoost/CatBoost%20Custom%20Prediction%20With%20FastAPI.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo">
      <br>Run in<br>Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2Fstatmike%2Fvertex-ai-mlops%2Fmain%2FFramework%2520Workflows%2FCatBoost%2FCatBoost%2520Custom%2520Prediction%2520With%2520FastAPI.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo">
      <br>Run in<br>Colab Enterprise
    </a>
  </td>      
  <td style="text-align: center">
    <a href="https://github.com/statmike/vertex-ai-mlops/blob/main/Framework%20Workflows/CatBoost/CatBoost%20Custom%20Prediction%20With%20FastAPI.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      <br>View on<br>GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/statmike/vertex-ai-mlops/main/Framework%20Workflows/CatBoost/CatBoost%20Custom%20Prediction%20With%20FastAPI.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      <br>Open in<br>Vertex AI Workbench
    </a>
  </td>
</table>

# CatBoost: Custom Prediction With FastAPI

Build a [custom container for prediction](https://cloud.google.com/vertex-ai/docs/predictions/custom-container-requirements) with [CatBoost](https://catboost.ai/) using [FastAPI](https://fastapi.tiangolo.com/) and store in [Artifact  Registry](https://cloud.google.com/artifact-registry/docs/overview) for enabling serving predictions from:
- Locally for testing (with Docker): on [Vertex AI Workbench Instances](https://cloud.google.com/vertex-ai/docs/workbench/instances/introduction) for example
- On [Cloud Run](https://cloud.google.com/run/docs/overview/what-is-cloud-run)
- On [Vertex AI Prediction Endpoints](https://cloud.google.com/vertex-ai/docs/general/deployment)

**Prerequisites:**
- [CatBoost Overview](./CatBoost%20Overview.ipynb)
    - Train the model used here and store it in GCS

---
## Colab Setup

To run this notebook in Colab run the cells in this section.  Otherwise, skip this section.

This cell will authenticate to GCP (follow prompts in the popup).

In [1]:
PROJECT_ID = 'statmike-mlops-349915' # replace with project ID

In [2]:
try:
    import google.colab
    from google.colab import auth
    auth.authenticate_user()
    !gcloud config set project {PROJECT_ID}
except Exception:
    pass

---
## Installs

The list `packages` contains tuples of package import names and install names.  If the import name is not found then the install name is used to install quitely for the current user.

In [3]:
# tuples of (import name, install name, min_version)
packages = [
    ('numpy', 'numpy'),
    ('catboost', 'catboost'),
    ('docker', 'docker'),
    ('google.cloud.aiplatform', 'google-cloud-aiplatform'),
    ('google.cloud.storage', 'google-cloud-storage'),
    ('google.cloud.artifactregistry_v1', 'google-cloud-artifact-registry'),
    ('google.cloud.devtools', 'google-cloud-build'),
    ('google.cloud.run_v2', 'google-cloud-run'),   
]

import importlib
install = False
for package in packages:
    if not importlib.util.find_spec(package[0]):
        print(f'installing package {package[1]}')
        install = True
        !pip install {package[1]} -U -q --user
    elif len(package) == 3:
        if importlib.metadata.version(package[0]) < package[2]:
            print(f'updating package {package[1]}')
            install = True
            !pip install {package[1]} -U -q --user

### API Enablement

In [4]:
!gcloud services enable artifactregistry.googleapis.com
!gcloud services enable cloudbuild.googleapis.com
!gcloud services enable run.googleapis.com

### Restart Kernel (If Installs Occured)

After a kernel restart the code submission can start with the next cell after this one.

In [5]:
if install:
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)
    IPython.display.display(IPython.display.Markdown("""<div class=\"alert alert-block alert-warning\">
        <b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. The previous cells do not need to be run again⚠️</b>
        </div>"""))

---
## Setup

inputs:

In [6]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'statmike-mlops-349915'

In [7]:
REGION = 'us-central1'
SERIES = 'frameworks'
EXPERIMENT = 'catboost-custom-container'

GCS_BUCKET = PROJECT_ID

packages:

In [8]:
import json, os
import time
import requests

import catboost 
import numpy as np
import docker

import google.auth
from google.cloud import storage
from google.cloud import artifactregistry_v1
from google.cloud.devtools import cloudbuild_v1
from google.cloud import run_v2
from google.cloud import aiplatform

clients:

In [9]:
# gcs storage client
gcs = storage.Client(project = GCS_BUCKET)
bucket = gcs.bucket(GCS_BUCKET)

# cloud build client
cb = cloudbuild_v1.CloudBuildClient()

# artifact registry client
ar = artifactregistry_v1.ArtifactRegistryClient()

# cloud run client
cr = run_v2.ServicesClient()

# vertex ai client
aiplatform.init(project = PROJECT_ID, location = REGION)

Parameters:

In [10]:
DIR = f"files/{EXPERIMENT}"

Environment:

In [11]:
if not os.path.exists(DIR):
    os.makedirs(DIR)

---
## CatBoost Model

Retrieve the model trained in prior workflow along with test records.  Test the model directly in this environment.

### Check For Files

In [12]:
files = list(bucket.list_blobs(prefix = f'{SERIES}/catboost-overview'))
if len(files) > 0:
    print('Found the files created by the prerequisite workflow:')
    for file in files:
        print(f'- gs://{bucket.name}/{file.name}')
else:
    print('Files note found - Please run the prerequisite notebook (listed at top of this workflow)')

Found the files created by the prerequisite workflow:
- gs://statmike-mlops-349915/frameworks/catboost-overview/examples.json
- gs://statmike-mlops-349915/frameworks/catboost-overview/model.cbm


### Load Model

In [13]:
model_blob = bucket.blob(f'{SERIES}/catboost-overview/model.cbm')
model_bytes = model_blob.download_as_bytes()
model = catboost.CatBoostClassifier().load_model(blob = model_bytes)

### Load Inference Examples

In [14]:
examples_blob = bucket.blob(f'{SERIES}/catboost-overview/examples.json')
examples_np = np.array(
    json.loads(examples_blob.download_as_string())
)

### Test Model With Examples

In [15]:
model.predict(examples_np)

array([0, 1, 0, 1, 1, 0, 1, 1, 0, 1])

---
## Build A Custom Prediction Container

It is really not all that hard with Python!

For this example [FastAPI](https://fastapi.tiangolo.com/) is used.

This process uses docker to build a custom container and then runs the container on Cloud Run and Vertex AI Endpoints.

This could be done locally with Docker and pushed to Artifact Registry before deployment to Cloud Run.  The process below assumes that docker is not available locally and used Cloud Build to both build and push the resulting container to Artifact Registry.

---
### Setup Artifact Registry

[Artifact registry](https://cloud.google.com/artifact-registry/docs) organizes artifacts with repositories.  Each repository contains packages and is designated to hold a partifcular format of package: Docker images, Python Packages and [others](https://cloud.google.com/artifact-registry/docs/supported-formats#package).

#### List Repositories

This may be empty if no repositories have been created for this project

In [16]:
for repo in ar.list_repositories(parent = f'projects/{PROJECT_ID}/locations/{REGION}'):
    print(repo.name)

projects/statmike-mlops-349915/locations/us-central1/repositories/frameworks
projects/statmike-mlops-349915/locations/us-central1/repositories/frameworks-catboost
projects/statmike-mlops-349915/locations/us-central1/repositories/gcf-artifacts
projects/statmike-mlops-349915/locations/us-central1/repositories/mlops
projects/statmike-mlops-349915/locations/us-central1/repositories/statmike-mlops-349915
projects/statmike-mlops-349915/locations/us-central1/repositories/statmike-mlops-349915-docker
projects/statmike-mlops-349915/locations/us-central1/repositories/statmike-mlops-349915-python


#### Create/Retrieve Docker Image Repository

Create an Artifact Registry Repository to hold Docker Images created by this notebook.  First, check to see if it is already created by a previous run and retrieve it if it has.  Otherwise, create one named for this project.

In [17]:
docker_repo = None
for repo in ar.list_repositories(parent = f'projects/{PROJECT_ID}/locations/{REGION}'):
    if f'{SERIES}' == repo.name.split('/')[-1]:
        docker_repo = repo
        print(f'Retrieved existing repo: {docker_repo.name}')

if not docker_repo:
    operation = ar.create_repository(
        request = artifactregistry_v1.CreateRepositoryRequest(
            parent = f'projects/{PROJECT_ID}/locations/{REGION}',
            repository_id = f'{SERIES}',
            repository = artifactregistry_v1.Repository(
                description = f'A repository for the {SERIES} series that holds docker images.',
                name = f'{SERIES}',
                format_ = artifactregistry_v1.Repository.Format.DOCKER,
                labels = {'series': SERIES}
            )
        )
    )
    print('Creating Repository ...')
    docker_repo = operation.result()
    print(f'Completed creating repo: {docker_repo.name}')

Retrieved existing repo: projects/statmike-mlops-349915/locations/us-central1/repositories/frameworks


In [18]:
docker_repo.name, docker_repo.format_.name

('projects/statmike-mlops-349915/locations/us-central1/repositories/frameworks',
 'DOCKER')

In [19]:
REPOSITORY = f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{docker_repo.name.split('/')[-1]}"

In [20]:
REPOSITORY

'us-central1-docker.pkg.dev/statmike-mlops-349915/frameworks'

---
### Create Application Files

```
|__ Dockerfile
|__ requirements.txt
|__ app
    |__ __init__.py
    |__ main.py
    |__ prestart.sh
```

In [21]:
if not os.path.exists(DIR + '/source/app'):
    os.makedirs(DIR + '/source/app')

In [22]:
%%writefile {DIR}/source/Dockerfile
FROM tiangolo/uvicorn-gunicorn-fastapi:python3.9

COPY ./app /app
COPY ./requirements.txt requirements.txt

RUN pip install --no-cache-dir --upgrade pip \
  && pip install --no-cache-dir -r requirements.txt

Overwriting files/catboost-custom-container/source/Dockerfile


In [23]:
%%writefile {DIR}/source/requirements.txt
google-cloud-storage
catboost
numpy

Overwriting files/catboost-custom-container/source/requirements.txt


In [24]:
%%writefile {DIR}/source/app/__init__.py
# init file

Overwriting files/catboost-custom-container/source/app/__init__.py


In [25]:
%%writefile {DIR}/source/app/main.py
# this version:
# - inputs to be json like {'instances': [[list],[list], ...]}
# - outputs in json like {'predictions': [[list],[list], ...]}
# trying to adhere to Vetex Endpoints Requirements:
# - https://cloud.google.com/vertex-ai/docs/predictions/get-online-predictions

# packages
import os
from fastapi import FastAPI, Request
import catboost
import numpy as np
from google.cloud import storage

# clients
app = FastAPI()
gcs = storage.Client()

# download the model file from GCS
paths = os.environ['AIP_STORAGE_URI'].split('/') + ['model.cbm']
bucket = gcs.bucket(paths[2])
blob = bucket.blob('/'.join(paths[3:]))
blob.download_to_filename('model.cbm')

# Load the catboost model
_model = catboost.CatBoostClassifier().load_model('model.cbm')

# get model classification levels
classes = [str(c) for c in list(_model.classes_)]

# Define function for health route
@app.get(os.environ['AIP_HEALTH_ROUTE'], status_code=200)
def health():
    return {}

# Define function for prediction route
@app.post(os.environ['AIP_PREDICT_ROUTE'])
async def predict(request: Request):
    # await the request
    body = await request.json()
    
    # parse the request
    instances = body["instances"]
    
    # get predicted probabilities
    predictions = _model.predict_proba(instances).tolist()    

    # this returns just the predicted probabilities:
    return {"predictions": predictions}

Overwriting files/catboost-custom-container/source/app/main.py


In [26]:
%%writefile {DIR}/source/app/main2.py
# this version:
# - inputs to be json like {'instances': [[list],[list], ...]}
# - outputs in json like {'predictions': [{'classes': list, 'scores': list, 'predicted_class': str}, ...]}
# trying to adhere to Vetex Endpoints Requirements:
# - https://cloud.google.com/vertex-ai/docs/predictions/get-online-predictions

# packages
import os
from fastapi import FastAPI, Request
import catboost
import numpy as np
from google.cloud import storage

# clients
app = FastAPI()
gcs = storage.Client()

# download the model file from GCS
paths = os.environ['AIP_STORAGE_URI'].split('/') + ['model.cbm']
bucket = gcs.bucket(paths[2])
blob = bucket.blob('/'.join(paths[3:]))
blob.download_to_filename('model.cbm')

# Load the catboost model
_model = catboost.CatBoostClassifier().load_model('model.cbm')

# get model classification levels
classes = [str(c) for c in list(_model.classes_)]

# Define function for health route
@app.get(os.environ['AIP_HEALTH_ROUTE'], status_code=200)
def health():
    return {}

# Define function for prediction route
@app.post(os.environ['AIP_PREDICT_ROUTE'])
async def predict(request: Request):
    # await the request
    body = await request.json()
    
    # parse the request
    instances = body["instances"]
    
    # get predicted probabilities
    predictions = _model.predict_proba(instances).tolist()
    
    # format predictions:
    preds = [dict(classes = classes, scores = p, predicted_class = classes[np.argmax(p)]) for p in predictions]
    
    # following outputs detail prediction info for classification:
    return {"predictions": preds}

Overwriting files/catboost-custom-container/source/app/main2.py


In [27]:
%%writefile {DIR}/source/app/prestart.sh
#!/bin/bash
export PORT=$AIP_HTTP_PORT

Overwriting files/catboost-custom-container/source/app/prestart.sh


In [28]:
bucket.blob(f'{SERIES}/{EXPERIMENT}/source/Dockerfile').upload_from_filename(f'{DIR}/source/Dockerfile')
bucket.blob(f'{SERIES}/{EXPERIMENT}/source/requirements.txt').upload_from_filename(f'{DIR}/source/requirements.txt')
bucket.blob(f'{SERIES}/{EXPERIMENT}/source/app/__init__.py').upload_from_filename(f'{DIR}/source/app/__init__.py')
bucket.blob(f'{SERIES}/{EXPERIMENT}/source/app/main.py').upload_from_filename(f'{DIR}/source/app/main.py')
bucket.blob(f'{SERIES}/{EXPERIMENT}/source/app/main2.py').upload_from_filename(f'{DIR}/source/app/main2.py')
bucket.blob(f'{SERIES}/{EXPERIMENT}/source/app/prestart.sh').upload_from_filename(f'{DIR}/source/app/prestart.sh')

In [29]:
list(bucket.list_blobs(prefix = f'{SERIES}/{EXPERIMENT}/source'))

[<Blob: statmike-mlops-349915, frameworks/catboost-custom-container/source/Dockerfile, 1739194583792229>,
 <Blob: statmike-mlops-349915, frameworks/catboost-custom-container/source/app/__init__.py, 1739194583946387>,
 <Blob: statmike-mlops-349915, frameworks/catboost-custom-container/source/app/main.py, 1739194584029001>,
 <Blob: statmike-mlops-349915, frameworks/catboost-custom-container/source/app/main2.py, 1739194584103663>,
 <Blob: statmike-mlops-349915, frameworks/catboost-custom-container/source/app/prestart.sh, 1739194584177041>,
 <Blob: statmike-mlops-349915, frameworks/catboost-custom-container/source/requirements.txt, 1739194583875840>]

---
### Build Application Container

Use the Cloud Build client to construct and run the build instructions. Here the files collected in GCS are copied to the build instance, then the Docker build is run in the folder with the `Dockerfile`. The resulting image is pushed to Artifact Registry (setup above).

In [30]:
# setup the build config with empty list of steps - these will be added sequentially
build = cloudbuild_v1.Build(
    steps = []
)
# retrieve the source
build.steps.append(
    {
        'name': 'gcr.io/cloud-builders/gsutil',
        'args': ['cp', '-r', f'gs://{GCS_BUCKET}/{SERIES}/{EXPERIMENT}/source/*', '/workspace']
    }
)
# docker build
build.steps.append(
    {
        'name': 'gcr.io/cloud-builders/docker',
        'args': ['build', '-t', f'{REPOSITORY}/{EXPERIMENT}', '/workspace']
    }    
)
# docker push
build.images = [f"{REPOSITORY}/{EXPERIMENT}"]

In [31]:
operation = cb.create_build(
    project_id = PROJECT_ID,
    build = build
)

In [32]:
build_response = operation.result()
build_response.status, build_response.artifacts

(<Status.SUCCESS: 3>,
 images: "us-central1-docker.pkg.dev/statmike-mlops-349915/frameworks/catboost-custom-container")

In [33]:
build_response.artifacts.images[0]

'us-central1-docker.pkg.dev/statmike-mlops-349915/frameworks/catboost-custom-container'

---
## Test Locally

If Docker is installed and running locally then use it to test the image.

In [34]:
try:
    local_test = True
    docker_client = docker.from_env()
    if docker_client.ping():
        print(f"Docker is installed and running. Version: {docker_client.version()['Version']}")
except Exception as e:
    local_test = False
    print('Docker is either not installed or not running - please fix before proceeding.\nhttps://docs.docker.com/engine/install/')

Docker is installed and running. Version: 20.10.17


### Pull and Run Container

Run the container image with:
- ports: inside 8080 mapped to outside 80
- set environment variables for:
    - `AIP_HTTP_PORT` is `8080`
    - `AIP_HEALTH_ROUTE` is `/health`
    - `AIP_PREDICT_ROUTE` is `/predict`
    - `AIP_STORAGE_URI` is the `gs://bucket/path/to/folder`
    - `MODULE_NAME` is 'main'
        - this actually defaults to main so is not required
        - an alternative script with different prediction output is created in `main2.py` above
        - use this environment variable to start the container using the alternative script in module `main2`
        - see the [FastAPI Docker Image Advanced Usage](https://github.com/tiangolo/uvicorn-gunicorn-fastapi-docker?tab=readme-ov-file#advanced-usage) details

In [46]:
if local_test:
    # make sure any prior runs are stopped:
    try:
        container = docker_client.containers.get('local-run')
        container.stop()
        container.remove()
    except docker.errors.NotFound:
        pass
    
    # get image:
    image_uri = build_response.artifacts.images[0]
    try:
        local_image = docker_client.images.get(image_uri)
        remote_image = docker_client.images.pull(image_uri)
        if local_image.id != remote_image.id:
            print('New image found, updating ...')
            local_image = remote_image
        else:
            print('Using existing image ...')
    except docker.errors.ImageNotFound:
        print('Pulling image ...')
        local_image = docker_client.images.pull(image_uri)
        
    # run container:
    print('Starting container ...')
    container = docker_client.containers.run(
        image = image_uri,
        detach = True,
        ports = {'8080/tcp':80}, # Map inside:outside (where docker run -p is outside:inside)
        name = 'local-run',
        environment = {
            'AIP_HTTP_PORT': '8080',
            'AIP_HEALTH_ROUTE': '/health',
            'AIP_PREDICT_ROUTE': '/predict',
            'AIP_STORAGE_URI': f'gs://{bucket.name}/{SERIES}/catboost-overview',
            'MODULE_NAME': 'main2' # try main2 for alternative output
        }
    )
    print('Container ready.\n\tUse `container.logs()` to view startup logs.')

Using existing image ...
Starting container ...
Container ready.
	Use `container.logs()` to view startup logs.


In [47]:
#container.logs()

### Health Check

Want to see `200`:

In [48]:
if local_test:
    response = requests.get(f"http://localhost:80/health")
    print(response.status_code)

200


### Inference Test

In [49]:
def predict(instances):
    url = f"http://localhost:80/predict"
    headers = {'Content_Type': 'application/json'}
    data = json.dumps({'instances': instances})
    response = requests.post(url, headers = headers, data = data)    
    return json.loads(response.text)

In [50]:
predict(examples_np[0:1].tolist())

{'predictions': [{'classes': ['0', '1'],
   'scores': [0.9684653609075246, 0.031534639092475496],
   'predicted_class': '0'}]}

In [51]:
predict(examples_np[1:2].tolist())

{'predictions': [{'classes': ['0', '1'],
   'scores': [0.004793214059859108, 0.9952067859401409],
   'predicted_class': '1'}]}

In [52]:
predict(examples_np[0:2].tolist())

{'predictions': [{'classes': ['0', '1'],
   'scores': [0.9684653609075246, 0.031534639092475496],
   'predicted_class': '0'},
  {'classes': ['0', '1'],
   'scores': [0.004793214059859108, 0.9952067859401409],
   'predicted_class': '1'}]}

In [53]:
predict(examples_np.tolist())

{'predictions': [{'classes': ['0', '1'],
   'scores': [0.9684653609075246, 0.031534639092475496],
   'predicted_class': '0'},
  {'classes': ['0', '1'],
   'scores': [0.004793214059859108, 0.9952067859401409],
   'predicted_class': '1'},
  {'classes': ['0', '1'],
   'scores': [0.9961659959204033, 0.0038340040795967153],
   'predicted_class': '0'},
  {'classes': ['0', '1'],
   'scores': [0.009687501567520629, 0.9903124984324794],
   'predicted_class': '1'},
  {'classes': ['0', '1'],
   'scores': [0.05002634033491404, 0.949973659665086],
   'predicted_class': '1'},
  {'classes': ['0', '1'],
   'scores': [0.9988754743912287, 0.0011245256087712965],
   'predicted_class': '0'},
  {'classes': ['0', '1'],
   'scores': [0.0058287633360615265, 0.9941712366639385],
   'predicted_class': '1'},
  {'classes': ['0', '1'],
   'scores': [0.0031326212134841214, 0.9968673787865159],
   'predicted_class': '1'},
  {'classes': ['0', '1'],
   'scores': [0.8122003166930434, 0.18779968330695668],
   'predicted

### Stop Container

In [54]:
container.name

'local-run'

In [55]:
container = docker_client.containers.get(container.name)

In [56]:
container.status

'running'

In [57]:
container.stop()
container.remove()

---
## Vertex AI Prediction Endpoint

Register the model in the [Vertex AI Model Registry](https://cloud.google.com/vertex-ai/docs/model-registry/introduction) and [Deploy it to an endpoint](https://cloud.google.com/vertex-ai/docs/general/deployment).

### Model Registry

Check for existing version of the model:

In [37]:
parent_model = ''
for model in aiplatform.Model.list(filter=f'display_name="{SERIES}"'):
    parent_model = model.resource_name
    break
parent_model

''

Upload the model to the registry with different versions for:
- plain responses - using main.py which is default
- formated responses - using main2.py which is set with environment variable

In [38]:
vertex_model = aiplatform.Model.upload(
    display_name = SERIES,
    model_id = SERIES,
    parent_model = parent_model,
    serving_container_image_uri = build_response.artifacts.images[0],
    artifact_uri = f'gs://{bucket.name}/{SERIES}/catboost-overview',
    is_default_version = True,
    version_aliases = [f'{EXPERIMENT}-plain'],
    version_description = EXPERIMENT,
    labels = {'series': SERIES, 'experiment': EXPERIMENT}
)

Creating Model
Create Model backing LRO: projects/1026793852137/locations/us-central1/models/frameworks/operations/982063891641008128
Model created. Resource name: projects/1026793852137/locations/us-central1/models/frameworks@1
To use this Model in another session:
model = aiplatform.Model('projects/1026793852137/locations/us-central1/models/frameworks@1')


In [39]:
vertex_model = aiplatform.Model.upload(
    parent_model = vertex_model.resource_name,
    serving_container_image_uri = build_response.artifacts.images[0],
    serving_container_environment_variables = {'MODULE_NAME': 'main2'},
    artifact_uri = f'gs://{bucket.name}/{SERIES}/catboost-overview',
    is_default_version = False,
    version_aliases = [f'{EXPERIMENT}-formatted'],
    version_description = EXPERIMENT,
    labels = {'series': SERIES, 'experiment': EXPERIMENT}
)

Creating Model
Create Model backing LRO: projects/1026793852137/locations/us-central1/models/frameworks/operations/1053699273213870080
Model created. Resource name: projects/1026793852137/locations/us-central1/models/frameworks@2
To use this Model in another session:
model = aiplatform.Model('projects/1026793852137/locations/us-central1/models/frameworks@2')


### Create Endpoint

Check for existing endpoint:

In [40]:
vertex_endpoint = None
for endpoint in aiplatform.Endpoint.list(filter=f'display_name="{SERIES}"'):
    vertex_endpoint = endpoint
    break
vertex_endpoint

Create endpoint if missing:

In [41]:
if not vertex_endpoint:
    vertex_endpoint = aiplatform.Endpoint.create(
        display_name = SERIES,
        labels = {'series': SERIES}   
    )
vertex_endpoint

Creating Endpoint
Create Endpoint backing LRO: projects/1026793852137/locations/us-central1/endpoints/8702565264614490112/operations/7345650165115518976
Endpoint created. Resource name: projects/1026793852137/locations/us-central1/endpoints/8702565264614490112
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/1026793852137/locations/us-central1/endpoints/8702565264614490112')


resource name: projects/1026793852137/locations/us-central1/endpoints/8702565264614490112

### Deploy Model: Default version with `main.py` - Plain

Get the latest model version with alias `plain`:

In [42]:
vertex_model = aiplatform.Model(model_name = SERIES, version = f'{EXPERIMENT}-plain')
vertex_model.versioned_resource_name

'projects/1026793852137/locations/us-central1/models/frameworks@1'

In [43]:
vertex_model.deploy(
    endpoint = vertex_endpoint,
    traffic_percentage = 100,
    machine_type = 'n1-standard-4',
    min_replica_count = 1,
    max_replica_count = 2,
)

Deploying model to Endpoint : projects/1026793852137/locations/us-central1/endpoints/8702565264614490112
Deploy Endpoint model backing LRO: projects/1026793852137/locations/us-central1/endpoints/8702565264614490112/operations/3035283109256888320
Endpoint model deployed. Resource name: projects/1026793852137/locations/us-central1/endpoints/8702565264614490112


resource name: projects/1026793852137/locations/us-central1/endpoints/8702565264614490112

### Test Predictions: Plain

In [44]:
vertex_endpoint.predict(instances = examples_np[0:1].tolist())

Prediction(predictions=[[0.9684653609075246, 0.0315346390924755]], deployed_model_id='3804641185104920576', metadata=None, model_version_id='1', model_resource_name='projects/1026793852137/locations/us-central1/models/frameworks', explanations=None)

In [45]:
vertex_endpoint.predict(instances = examples_np[1:2].tolist()).predictions

[[0.004793214059859108, 0.9952067859401409]]

In [46]:
vertex_endpoint.predict(instances = examples_np[0:2].tolist()).predictions

[[0.9684653609075246, 0.0315346390924755],
 [0.004793214059859108, 0.9952067859401409]]

In [47]:
vertex_endpoint.predict(instances = examples_np.tolist()).predictions

[[0.9684653609075246, 0.0315346390924755],
 [0.004793214059859108, 0.9952067859401409],
 [0.9961659959204033, 0.003834004079596715],
 [0.009687501567520629, 0.9903124984324794],
 [0.05002634033491404, 0.949973659665086],
 [0.9988754743912287, 0.001124525608771297],
 [0.005828763336061527, 0.9941712366639385],
 [0.003132621213484121, 0.9968673787865159],
 [0.8122003166930434, 0.1877996833069567],
 [0.007672836590005838, 0.9923271634099942]]

### Deploy Model: Version with `main2.py` - Formatted

In [48]:
vertex_model = aiplatform.Model(model_name = SERIES, version = f'{EXPERIMENT}-formatted')
vertex_model.versioned_resource_name

'projects/1026793852137/locations/us-central1/models/frameworks@2'

In [49]:
vertex_model.deploy(
    endpoint = vertex_endpoint,
    traffic_percentage = 0,
    machine_type = 'n1-standard-4',
    min_replica_count = 1,
    max_replica_count = 2,
)

Deploying model to Endpoint : projects/1026793852137/locations/us-central1/endpoints/8702565264614490112
Deploy Endpoint model backing LRO: projects/1026793852137/locations/us-central1/endpoints/8702565264614490112/operations/5033755443902545920
Endpoint model deployed. Resource name: projects/1026793852137/locations/us-central1/endpoints/8702565264614490112


resource name: projects/1026793852137/locations/us-central1/endpoints/8702565264614490112

### Shift Traffic To Model Version: Formatted

In [50]:
vertex_endpoint.traffic_split

{'3804641185104920576': 100, '772029786024312832': 0}

In [51]:
new_traffic_split = {}
for deployed_model in vertex_endpoint.list_models():
    if deployed_model.model_version_id == vertex_model.version_id:
        new_traffic_split[deployed_model.id] = 100
    else:
        new_traffic_split[deployed_model.id] = 0
new_traffic_split

{'772029786024312832': 100, '3804641185104920576': 0}

In [52]:
vertex_endpoint.update(traffic_split = new_traffic_split)

Updating Endpoint endpoint: projects/1026793852137/locations/us-central1/endpoints/8702565264614490112
Endpoint endpoint updated. Resource name: projects/1026793852137/locations/us-central1/endpoints/8702565264614490112


resource name: projects/1026793852137/locations/us-central1/endpoints/8702565264614490112

In [53]:
vertex_endpoint.traffic_split

{'3804641185104920576': 0, '772029786024312832': 100}

### Test Predictions: Formatted

In [54]:
vertex_endpoint.predict(instances = examples_np[0:1].tolist())

Prediction(predictions=[{'predicted_class': '0', 'classes': ['0', '1'], 'scores': [0.9684653609075246, 0.0315346390924755]}], deployed_model_id='772029786024312832', metadata=None, model_version_id='2', model_resource_name='projects/1026793852137/locations/us-central1/models/frameworks', explanations=None)

In [55]:
vertex_endpoint.predict(instances = examples_np[1:2].tolist()).predictions

[{'predicted_class': '1',
  'classes': ['0', '1'],
  'scores': [0.004793214059859108, 0.9952067859401409]}]

In [56]:
vertex_endpoint.predict(instances = examples_np[0:2].tolist()).predictions

[{'predicted_class': '0',
  'classes': ['0', '1'],
  'scores': [0.9684653609075246, 0.0315346390924755]},
 {'classes': ['0', '1'],
  'predicted_class': '1',
  'scores': [0.004793214059859108, 0.9952067859401409]}]

In [57]:
vertex_endpoint.predict(instances = examples_np.tolist()).predictions

[{'predicted_class': '0',
  'classes': ['0', '1'],
  'scores': [0.9684653609075246, 0.0315346390924755]},
 {'predicted_class': '1',
  'classes': ['0', '1'],
  'scores': [0.004793214059859108, 0.9952067859401409]},
 {'predicted_class': '0',
  'classes': ['0', '1'],
  'scores': [0.9961659959204033, 0.003834004079596715]},
 {'predicted_class': '1',
  'classes': ['0', '1'],
  'scores': [0.009687501567520629, 0.9903124984324794]},
 {'classes': ['0', '1'],
  'predicted_class': '1',
  'scores': [0.05002634033491404, 0.949973659665086]},
 {'predicted_class': '0',
  'classes': ['0', '1'],
  'scores': [0.9988754743912287, 0.001124525608771297]},
 {'classes': ['0', '1'],
  'predicted_class': '1',
  'scores': [0.005828763336061527, 0.9941712366639385]},
 {'classes': ['0', '1'],
  'predicted_class': '1',
  'scores': [0.003132621213484121, 0.9968673787865159]},
 {'classes': ['0', '1'],
  'predicted_class': '0',
  'scores': [0.8122003166930434, 0.1877996833069567]},
 {'classes': ['0', '1'],
  'predic

### Shift Traffic To Model: Plain

In [58]:
vertex_endpoint.traffic_split

{'3804641185104920576': 0, '772029786024312832': 100}

In [59]:
new_traffic_split = {}
for deployed_model in vertex_endpoint.list_models():
    if deployed_model.model_version_id != vertex_model.version_id:
        new_traffic_split[deployed_model.id] = 100
    else:
        new_traffic_split[deployed_model.id] = 0
vertex_endpoint.update(traffic_split = new_traffic_split)

Updating Endpoint endpoint: projects/1026793852137/locations/us-central1/endpoints/8702565264614490112
Endpoint endpoint updated. Resource name: projects/1026793852137/locations/us-central1/endpoints/8702565264614490112


resource name: projects/1026793852137/locations/us-central1/endpoints/8702565264614490112

In [60]:
vertex_endpoint.traffic_split

{'3804641185104920576': 100, '772029786024312832': 0}

In [61]:
vertex_endpoint.predict(instances = examples_np[0:2].tolist()).predictions

[[0.9684653609075246, 0.0315346390924755],
 [0.004793214059859108, 0.9952067859401409]]

### Undeploy Models Without Traffic

In [62]:
for deployed_model in vertex_endpoint.list_models():
    if vertex_endpoint.traffic_split[deployed_model.id] == 0:
        vertex_endpoint.undeploy(deployed_model_id = deployed_model.id)
vertex_endpoint.traffic_split

Undeploying Endpoint model: projects/1026793852137/locations/us-central1/endpoints/8702565264614490112
Undeploy Endpoint model backing LRO: projects/1026793852137/locations/us-central1/endpoints/8702565264614490112/operations/2860064936254504960
Endpoint model undeployed. Resource name: projects/1026793852137/locations/us-central1/endpoints/8702565264614490112


{'3804641185104920576': 100}

### Undeploy All Models

In [63]:
for deployed_model in vertex_endpoint.list_models():
    vertex_endpoint.undeploy(deployed_model_id = deployed_model.id)
vertex_endpoint.list_models()

Undeploying Endpoint model: projects/1026793852137/locations/us-central1/endpoints/8702565264614490112
Undeploy Endpoint model backing LRO: projects/1026793852137/locations/us-central1/endpoints/8702565264614490112/operations/3224434293606449152
Endpoint model undeployed. Resource name: projects/1026793852137/locations/us-central1/endpoints/8702565264614490112


[]

### Delete Endpoint

In [64]:
vertex_endpoint.delete(force = True)

Deleting Endpoint : projects/1026793852137/locations/us-central1/endpoints/8702565264614490112
Endpoint deleted. . Resource name: projects/1026793852137/locations/us-central1/endpoints/8702565264614490112
Deleting Endpoint resource: projects/1026793852137/locations/us-central1/endpoints/8702565264614490112
Delete Endpoint backing LRO: projects/1026793852137/locations/us-central1/operations/6773270799974400000
Endpoint resource projects/1026793852137/locations/us-central1/endpoints/8702565264614490112 deleted.


---
## Cloud Run

Deploy the model to [Cloud Run](https://cloud.google.com/run/docs/overview/what-is-cloud-run) using the same container build and tested above from Artifact Registry.

Some highlights for Cloud Run:
- Rapid scaling to handle requests
- Scale to zero (default) or other minimum if set
- Can handle larger input (request) and output (response) sizes
    - See [requests limits](https://cloud.google.com/run/quotas#request_limits)
- Configure [memory limits](https://cloud.google.com/run/docs/configuring/services/memory-limits) and [cpu limits](https://cloud.google.com/run/docs/configuring/services/cpu) and [concurrency](https://cloud.google.com/run/docs/configuring/concurrency) and [autoscaling](https://cloud.google.com/run/docs/about-instance-autoscaling) and [request timeout](https://cloud.google.com/run/docs/configuring/request-timeout)

### Deploy The Endpoint

In [83]:
parent = f"projects/{PROJECT_ID}/locations/{REGION}"
service = run_v2.Service()
#service.name = f"{parent}/services/{SERIES}"
service.template.containers = [
    run_v2.Container(
        image = build_response.artifacts.images[0],
        ports = [run_v2.ContainerPort(container_port = 8080)],
        env = [
            run_v2.EnvVar(name = 'AIP_HTTP_PORT', value = '8080'),
            run_v2.EnvVar(name = 'AIP_HEALTH_ROUTE', value = '/health'),
            run_v2.EnvVar(name = 'AIP_PREDICT_ROUTE', value = '/predict'),
            run_v2.EnvVar(name = 'AIP_STORAGE_URI', value = f'gs://{bucket.name}/{SERIES}/catboost-overview'),
            run_v2.EnvVar(name = 'MODULE_NAME', value = 'main2')
        ],
        resources = run_v2.ResourceRequirements(
            limits = {"cpu": '8', "memory": '32Gi'}
        )
    )
]
service.ingress = run_v2.IngressTraffic.INGRESS_TRAFFIC_INTERNAL_ONLY

In [84]:
try:
    # create the service:
    run_response = cr.create_service(request = {"parent": parent, "service": service, "service_id": SERIES})
    # wait on the operation to complete:
    run_response.result()
    # print the name of the service
    print(f"Started Service: {run_response.metadata.name}")
except Exception as e:
    print(f"Error creating service: {e}")

Started Service: projects/statmike-mlops-349915/locations/us-central1/services/frameworks


In [85]:
run_response.metadata.uri

'https://frameworks-urlxi72dpa-uc.a.run.app'

### Permissions

The endpoint requires authentication.  Check ou tthe [Authentication Overview](https://cloud.google.com/run/docs/authenticating/overview) and in the case below the [Authenticating service-to-service](https://cloud.google.com/run/docs/authenticating/service-to-service) method is used by giving the same service account used to run the notebook and create the endpoint the role to invoke the endpoint as well.

In [86]:
SERVICE_ACCOUNT = !gcloud config list --format='value(core.account)' 
SERVICE_ACCOUNT = SERVICE_ACCOUNT[0]
SERVICE_ACCOUNT

'1026793852137-compute@developer.gserviceaccount.com'

In [87]:
run_response.metadata.name

'projects/statmike-mlops-349915/locations/us-central1/services/frameworks'

In [88]:
policy = cr.get_iam_policy(request = {'resource': run_response.metadata.name})
policy.bindings.add(
    role = 'roles/run.invoker',
    members = [f"serviceAccount:{SERVICE_ACCOUNT}", 'allUsers'] #'allUsers'
)
policy_response = cr.set_iam_policy(request = {"resource": run_response.metadata.name, "policy": policy})
print(f"IAM policy updated: {policy_response.bindings}")

IAM policy updated: [role: "roles/run.invoker"
members: "allUsers"
members: "serviceAccount:1026793852137-compute@developer.gserviceaccount.com"
]


In [89]:
policy.bindings

[role: "roles/run.invoker"
members: "serviceAccount:1026793852137-compute@developer.gserviceaccount.com"
members: "allUsers"
]

**WAIT: The update of the IAM Policy might take a few moments to take effect.  Rerun the following health check section until you get a `200` response code.**

### Health Check

Want to see `200`:

In [90]:
def health(uri):
    url = f"{uri}/health"
    credentials, _ = google.auth.default()
    auth_req = google.auth.transport.requests.Request()
    credentials.refresh(auth_req)
    headers = {'Authorization': f'Bearer {credentials.token}'}
    response = requests.get(url, headers = headers)    
    return response.status_code

def check_health(uri, timeout_seconds = 200, retry_seconds = 10):
    start_time = time.time()
    while True:
        status_code = health(uri)
        if status_code == 200:
            break
        elapsed_time = time.time() - start_time
        if elapsed_time > timeout_seconds:
            break
        time.sleep(retry_seconds)
    return status_code

In [91]:
check_health(run_response.metadata.uri)

200

In [92]:
health(run_response.metadata.uri)

200

### Inference Test

In [93]:
def predict(instances):
    credentials, _ = google.auth.default()
    auth_req = google.auth.transport.requests.Request()
    credentials.refresh(auth_req)
    url = f"{run_response.metadata.uri}/predict"
    headers = {
        'Authorization': f'Bearer {credentials.token}',
        'Content_Type': 'application/json'
    }
    data = json.dumps({'instances': instances})
    response = requests.post(url, headers = headers, data = data)    
    return json.loads(response.text)

In [94]:
predict(examples_np[0:1].tolist())

{'predictions': [{'classes': ['0', '1'],
   'scores': [0.9684653609075246, 0.031534639092475496],
   'predicted_class': '0'}]}

In [95]:
predict(examples_np[1:2].tolist())

{'predictions': [{'classes': ['0', '1'],
   'scores': [0.004793214059859108, 0.9952067859401409],
   'predicted_class': '1'}]}

In [96]:
predict(examples_np[0:2].tolist())

{'predictions': [{'classes': ['0', '1'],
   'scores': [0.9684653609075246, 0.031534639092475496],
   'predicted_class': '0'},
  {'classes': ['0', '1'],
   'scores': [0.004793214059859108, 0.9952067859401409],
   'predicted_class': '1'}]}

In [97]:
predict(examples_np.tolist())

{'predictions': [{'classes': ['0', '1'],
   'scores': [0.9684653609075246, 0.031534639092475496],
   'predicted_class': '0'},
  {'classes': ['0', '1'],
   'scores': [0.004793214059859108, 0.9952067859401409],
   'predicted_class': '1'},
  {'classes': ['0', '1'],
   'scores': [0.9961659959204033, 0.0038340040795967153],
   'predicted_class': '0'},
  {'classes': ['0', '1'],
   'scores': [0.009687501567520629, 0.9903124984324794],
   'predicted_class': '1'},
  {'classes': ['0', '1'],
   'scores': [0.05002634033491404, 0.949973659665086],
   'predicted_class': '1'},
  {'classes': ['0', '1'],
   'scores': [0.9988754743912287, 0.0011245256087712965],
   'predicted_class': '0'},
  {'classes': ['0', '1'],
   'scores': [0.0058287633360615265, 0.9941712366639385],
   'predicted_class': '1'},
  {'classes': ['0', '1'],
   'scores': [0.0031326212134841214, 0.9968673787865159],
   'predicted_class': '1'},
  {'classes': ['0', '1'],
   'scores': [0.8122003166930434, 0.18779968330695668],
   'predicted

### Remove The Service

Cloud Run will scale to zero here since a minimum has not been set.  This notebook does proceed with deleting the service in the following code.

In [98]:
remove_response = cr.delete_service(request = {"name": run_response.metadata.name})

In [99]:
remove_response.result()

name: "projects/statmike-mlops-349915/locations/us-central1/services/frameworks"
uid: "3a9767e1-5011-46a2-ba47-30abe9af6895"
generation: 2
create_time {
  seconds: 1739146275
  nanos: 167519000
}
update_time {
  seconds: 1739146433
  nanos: 950767000
}
delete_time {
  seconds: 1739146432
  nanos: 767411000
}
expire_time {
  seconds: 1741738432
  nanos: 767411000
}
creator: "1026793852137-compute@developer.gserviceaccount.com"
last_modifier: "1026793852137-compute@developer.gserviceaccount.com"
ingress: INGRESS_TRAFFIC_INTERNAL_ONLY
launch_stage: GA
template {
  scaling {
    max_instance_count: 100
  }
  timeout {
    seconds: 300
  }
  service_account: "1026793852137-compute@developer.gserviceaccount.com"
  containers {
    image: "us-central1-docker.pkg.dev/statmike-mlops-349915/frameworks/catboost-custom-container"
    env {
      name: "AIP_HTTP_PORT"
      value: "8080"
    }
    env {
      name: "AIP_HEALTH_ROUTE"
      value: "/health"
    }
    env {
      name: "AIP_PREDICT_R